In [1]:
import os
import shutil
import numpy as np
from skimage import io
from PIL import Image
from sklearn.cluster import KMeans

In [3]:
with open(f'{workdirectory}/cfar/pick_list.txt', 'r', encoding='utf-8') as f:
    picked = []
    for line in f:
        picked.append(line.strip())

In [4]:
picked

['D:/whale/data/1805/190402',
 'D:/whale/data/1805/180820',
 'D:/whale/data/1805/190407',
 'D:/whale/data/1805/181224',
 'D:/whale/data/1805/190725']

In [5]:
fig_dir = f'D:/whale/data/{obs}/180820/fig'
conv_dir = f'D:/whale/data/{obs}/180820/fig_convert'
group_dir = f'D:/whale/data/{obs}/180820/fig_group'

In [6]:
os.makedirs(fig_dir, exist_ok=True)
os.makedirs(conv_dir, exist_ok=True)

In [12]:
for pick in picked:
    files = os.listdir(f'{pick}/fig')
    
    for file in files:
        shutil.move(f'{pick}/fig/{file}',fig_dir)

Error: Destination path 'D:/whale/data/1805/180820/fig\180820_0.jpg' already exists

画像を200×200にリサイズ。元画像のままだと大きすぎてエラーになったため。

In [7]:
for path in os.listdir(fig_dir):
    img = Image.open(f'{fig_dir}/{path}')
    img = img.convert('RGB')
    img_resize = img.resize((200, 200))
    img_resize.save(f'{conv_dir}/{path}.jpg')

画像データは３次元配列のため、２次元配列に変換する。(200,200,3)から(200,600)

In [8]:
feature = np.array([io.imread(f'{conv_dir}/{path}') for path in os.listdir(conv_dir)])
feature = feature.reshape(len(feature), -1).astype(np.float64)

k平均法でクラスタリング
５つのクラスに分類する。ナガスクジラの鳴音が５種類あったため。
しかし少ないクラス数で一回鳴音以外をはじいた方がいいかも

In [ ]:
model = KMeans(n_clusters=5).fit(feature)

学習結果のラベルを取得する。0-4で振り分けられており、同じ数字であれば類似度が高い。

In [ ]:
labels = model.labels_

In [ ]:
for label, path in zip(labels, os.listdir(conv_dir)):
    os.makedirs(f'{group_dir}/{label}', exist_ok=True)
    shutil.copyfile(f"{conv_dir}/{path}", f"{group_dir}/{label}/{path.replace('.jpg', '',1)}")